## I. Importing data

In [1]:
import pandas as pd
import numpy as np
data_x = pd.read_csv('traininginputs.csv')
data_y = pd.read_csv('trainingoutput.csv')

## II. Cleaning data 

In [2]:
data = pd.merge(data_x, data_y, on='PROC_TRACEINFO', how ='inner')

Nous explorant notre base de donnée afin de se faire une idée sur son contenu. Pour cela commencons par voir les dimensions de notre dataset (nombre d'observations et de variables):

In [3]:
nbr_lignes, nbr_colonnes = data.shape
print("Nbr de lignes :", nbr_lignes )
print("Nbr de colonnes :",nbr_colonnes )

Nbr de lignes : 34515
Nbr de colonnes : 15


Nous avons donc 14 variables pour 34.515 individus. 

Vérifions s'il y a des valeurs manquantes.

In [4]:
data.isna().sum()

PROC_TRACEINFO                         0
OP070_V_1_angle_value                  0
OP090_SnapRingPeakForce_value          0
OP070_V_2_angle_value                  0
OP120_Rodage_I_mesure_value            0
OP090_SnapRingFinalStroke_value        0
OP110_Vissage_M8_torque_value          0
OP100_Capuchon_insertion_mesure    18627
OP120_Rodage_U_mesure_value            0
OP070_V_1_torque_value                 0
OP090_StartLinePeakForce_value         0
OP110_Vissage_M8_angle_value           0
OP090_SnapRingMidPointForce_val        0
OP070_V_2_torque_value                 0
Binar OP130_Resultat_Global_v          0
dtype: int64

Il semblerait que la variable 'OP100_Capuchon_insertion_mesure' compte 18.627 valeurs manquantes soit pour 53% des observations. Dans ce cas il serait difficile et pas pertinant de remplacer ces valeurs manquantes. Deux options s'offrirons a nous : utiliser des modèles qui gèrent les valeurs manquantes ou tout simplement supprimer cette variable. Les deux options seront explorées.

In [5]:
type_col = data.dtypes
print(type_col)

PROC_TRACEINFO                      object
OP070_V_1_angle_value              float64
OP090_SnapRingPeakForce_value      float64
OP070_V_2_angle_value              float64
OP120_Rodage_I_mesure_value        float64
OP090_SnapRingFinalStroke_value    float64
OP110_Vissage_M8_torque_value      float64
OP100_Capuchon_insertion_mesure    float64
OP120_Rodage_U_mesure_value        float64
OP070_V_1_torque_value             float64
OP090_StartLinePeakForce_value     float64
OP110_Vissage_M8_angle_value       float64
OP090_SnapRingMidPointForce_val    float64
OP070_V_2_torque_value             float64
Binar OP130_Resultat_Global_v        int64
dtype: object


La variable PROC_TRACEINFO est sous format 'object' et n a aucun interet. L énnoncé confirme qu il s agit Id de pieces. Nous décidons donc de la supprimer:

In [6]:
data = data.drop('PROC_TRACEINFO', axis =1 )

OK toutes les variables sont quantitatives. 
Par contre notre prédicteur (Binar OP130_Resultat_Global_v) devrait etre une variable catégorielle a deux modalités et non une variable continue. Nous procédons donc au changement de son type.

In [7]:
data['Binar OP130_Resultat_Global_v'] = data['Binar OP130_Resultat_Global_v'].astype('category')
print(data['Binar OP130_Resultat_Global_v'].dtype)

category


## III. Data analysis 

L'analyse de donnée est un travail déterminant pour la bonne compréhension de nos données et permet parfois de mieux aiguiller le travail de modélisation. L'analyse de données permet d'étudier aussi bien la relation entre les différentes variables mais aussi de se faire une idée sur la population (exemple son homogénéité). Dans le cas ou l'on a une population hétérogène, il serait pertinant de créer plusieurs groupes d'observations, dans ce cas deux solutions sont envisageables : développer un modèle par groupe homoène d'invidus , ou encore rajouter une variable dans notre base de donnée qui porterait le groupe auquel appartiendrait chaque observation. L'inconvénient de cette stratégie est que pour chaque nouvelle observation , il faudra tout d'abord déterminer le groupe auquel elle appartient avant de l'intégrer dans le modèle.

In [8]:
print(data.describe())

       OP070_V_1_angle_value  OP090_SnapRingPeakForce_value  \
count           34515.000000                   34515.000000   
mean              159.906922                     156.915055   
std                15.662650                      11.271492   
min               101.800000                       0.000000   
25%               148.700000                     149.210000   
50%               158.000000                     156.180000   
75%               169.300000                     164.380000   
max               198.300000                     196.920000   

       OP070_V_2_angle_value  OP120_Rodage_I_mesure_value  \
count           34515.000000                 34515.000000   
mean              159.618236                   113.350222   
std                15.091490                     3.528522   
min                82.000000                    99.990000   
25%               149.400000                   111.040000   
50%               158.700000                   113.160000   
75%  

Contenu de la présence de valeurs manquantes dans l une de nos varibles nous décidons de partir un XGboost directement en premier essai car il intégre la possibilité de gestion de valeurs manquantes.

# VI. Equilibrage du prédicteur ( to do)

StratifiedKFold est une variation de la validation croisée k-fold qui garantit que chaque pli (fold) a une distribution similaire des classes (ou étiquettes) de la variable cible par rapport à l'ensemble de données d'origine. Cette technique est particulièrement utile lorsque vous travaillez avec des ensembles de données déséquilibrés, 

# IV. ML modeling

#### 1. Diviser les données en train/test et définition des paramètres de la Validation croisée

In [9]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import make_scorer, roc_auc_score

Séparer les données en prédicteur (y) / variables prédictives (X) :

In [10]:
y = data['Binar OP130_Resultat_Global_v']
X = data.drop('Binar OP130_Resultat_Global_v', axis =1)

Nous séparons notre dataset en train/test en choisissant d'allouer 20% aux données test.  

In [11]:
X_train, X_test, y_train, y_test= train_test_split(X, y,test_size=0.2, random_state=123)

#### 2.1 Définition du modèle XG Boost

XGBoost, en tant qu'algorithme de boosting basé sur des arbres de décision, n'a  pas besoin de centrer ou de réduire les données en amont. Nous procédons donc au paramètrage de notre modèle. 
Comme évoqué précédemment, XGBoost est capable de gérer les valeurs manquantes automatiquement. Aucune configuration ne sera donc necessaire pour la gestion des valeurs manquantes.

In [12]:

model = xgb.XGBClassifier(
    n_jobs = -1, #utilise tous les coeurs CPU
    booster = 'gbtree',  # utilise les arbres de décision pour construire l'ensemble
    objective = 'binary:logistic', #classification a 2 classes (binaire)
    eval_metric = 'auc', 
    enable_categorical = True, 
    verbosity = 1 # printing messages , 0 (silent), 1 (warning), 2 (info), and 3 (debug). 
)

#### 2.2 Recherche d'hyperparamètres

Il existe deux types de paramètres en XGBoost : les paramètres intervenant lors du calcul du gain ou interet d ajouter un nouvel étage et donc infulancant la structure de l arbre  & ceux impactant le calcul du poids optimal a chaque feuille d abre impact donc directement les prédictions.

##### Option 1: Recherche avec Grille (avec GridSearchCV)

GridSearchCV effectue une recherche exhaustive à travers toutes les combinaisons possibles d'hyperparamètres spécifiées. Il parcourt chaque combinaison dans une grille prédéfinie, ce qui signifie qu'il teste toutes les valeurs d'hyperparamètres pour chaque hyperparamètre, ce qui peut être très coûteux en termes de calcul. Dans ce cas, nous nous contentons de selectionner aléatoirement quelques valeurs:  

In [13]:
# Spécifiez la grille des hyperparamètres à explorer
param_grid = {
    'learning_rate': [ 0.05 ,0.06], # fraction de la correction appliquée
    'n_estimators': [50, 100, 120], # nombre d arbres a entrainer séquentiellement pour faire prédicteur final (biais /variance)
    'max_depth': [3, 4, 5], #profondeur maximale de ce que peut atteindre un arbre et donc des valeurs qu'il peut prendre soit 2^max_depth
    'min_child_weight': [1, 2, 3],
    'gamma': [ 0.1, 0.2], #définit le seuil du gain qu'apporte l'ajou de chaque nouveau noeud (0 -> pas de seuil)
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 0.7],
}

# Créez un objet GridSearchCV pour la recherche par grille
grid_search = GridSearchCV(model, param_grid, scoring='roc_auc', cv=5, verbose=10, n_jobs=-1)

# Entraînez le modèle en utilisant la recherche par grille
grid_search.fit(X_train, y_train)

# Affichez les meilleurs hyperparamètres trouvés
print("Meilleurs hyperparamètres:", grid_search.best_params_)


Fitting 5 folds for each of 972 candidates, totalling 4860 fits
[CV 4/5; 1/972] START colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, subsample=0.8
[CV 1/5; 1/972] START colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, subsample=0.8
[CV 2/5; 2/972] START colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, subsample=0.9
[CV 2/5; 1/972] START colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, subsample=0.8
[CV 1/5; 2/972] START colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, subsample=0.9
[CV 3/5; 1/972] START colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=3, min_child_weight=1, n_estimators=50, subsample=0.8
[CV 5/5; 1/972] START colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=3, min_child_weig

KeyboardInterrupt: 

In [ ]:
best_params = grid_search.best_params_ #enregistre les meilleurs paramètres trouvés.

best_model = xgb.XGBClassifier(**best_params)  # Remplacez XGBClassifier par le modèle que vous utilisez

best_model.fit(X_train, y_train) #entrainer le modèle avec les meilleurs paramètres trouvés.

y_pred_proba = best_model.predict_proba(X_test)[:, 1]  # prédire sur la partie X_test par vos données de test

roc_auc = roc_auc_score(y_test, y_pred_proba)  # Remplacez y_test par vos étiquettes de test

print("ROC AUC Score:", roc_auc)

ROC AUC Score: 0.7066849126907246


Avec cette approche nous arrivons au score 0.70 ROC_AUC ce qui est suppérieur au score de 0.675 obtenu par Valéo en utilisant une   classification naïve bayésienne. 

Pour l'obtention de ce score, les meilleurs hyperparamètres sont :  {'colsample_bytree': 0.8, 'gamma': 0.1, 'learning_rate': 0.06, 'max_depth': 4, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 1.0}

Néanmoins, l'inconvénient de l'approche est qu'il est très probable quelle soit biaisée par l'invention de l'humain. Le champs des paramètres possibles étant très large , le risque de tomber sur un 'minimal local' est donc très important.

##### Option 2 : Recherche aléatoire (avec RandomizedSearchCV)

Randomized Search sélectionne aléatoirement un nombre spécifié de combinaisons d'hyperparamètres à partir de l'espace d'hyperparamètres. Il effectue une recherche aléatoire parmi un sous-ensemble d'hyperparamètres, ce qui peut être plus efficace que Grid Search en termes de temps de calcul.

L'approche est de définir un nombre fixe de combinaison a tester , RandomizedSearchCV selectionne aléatoirement les valeurs pour chaque hyperparamètre a chaque itération. Cette approche a plusieurs avantanges : 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [ ]:
param_dist = { #définition de l espace de recherche  
    'max_depth': range(3, 10),
    'learning_rate': [0.01, 0.02, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6],
    'n_estimators': range(50, 200),
    'subsample': [0.5, 0.7, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.7, 0.8, 1.0],
    'min_child_weight': range(1, 10),
    'gamma': [0, 0.1, 0.2, 0.3, 0.4],
}


In [ ]:
# Créer RandomizedSearchCV
random_search = RandomizedSearchCV(
    model,
    param_distributions=param_dist,
    n_iter=5000,  # Nombre d'itérations d'optimisation
    cv=5,  # Nombre pour la validation croisée
    scoring='roc_auc',  # Métrique de performance (ROC AUC)
    verbose=2,  # Afficher des détails pendant la recherche
    n_jobs=-1,  # Utiliser tous les cœurs du CPU
    random_state=42,
)

# Exécuter la recherche d'hyperparamètres
random_search.fit(X_train, y_train)

In [ ]:
# Afficher les meilleurs hyperparamètres trouvés
best_params = random_search.best_params_
print("Meilleurs hyperparamètres:", best_params)
print("Best score = %.3f after %d runs" % (random_search.best_score_, random_search.n_iter))

Meilleurs hyperparamètres: {'subsample': 1.0, 'n_estimators': 84, 'min_child_weight': 5, 'max_depth': 9, 'learning_rate': 0.01, 'gamma': 0.2, 'colsample_bytree': 0.5}
Best score = 0.658 after 5000 runs


Best score = 0.662 after 2000 runs

Best score = 0.662 after 4000 runs

Best score = 0.658 after 4000 runs

Best score = 0.658 after 5000 runs 23min

##### Option 3: Processus bayesien (BayesSearchCV de Scikit-optimize)

L’idée  de l’optimisation bayésienne est de minimiser le nombre d’observations tout en convergeant rapidement vers la solution optimale. Concrétement l'algorithme déterminerait la prochaine configuration ayant le plus de potentiel à tester en fonction des résultats des itérations précédentes en s'appuyant sur un processus gaussien.

Pour cela, il convient de connaître trois principes fondamentaux:
- Le processus Gaussien : d’exploiter les observations connues pour en déduire des probabilités d’événement qui n’ont pas encore été observées. Pour cela, il convient de déterminer pour chaque valeur X la distribution de probabilité. Le processus n'est pas applicable a toutes les observations a cause des limites de capacités calculatoires.

- Déterminer les points à plus fort potentiel avec la fonction d’acquisition : gagner en connaissance sur le comportement de la fonction et donc choisir une zone de l’espace de recherche où l’inconnu est grand : c’est l’exploration. D’autre part, nous souhaitons trouver le point qui minimise/maximise notre fonction : c’est l’exploitation. Ce compromis entre exploration et exploitation est exprimé par une fonction d’acquisition

- La fonction d’acquisition : Deux raisons permettent à une configuration donnée d’augmenter son potentiel, soit être dans une région loin de toutes configurations testées précédemment ou être dans une région près d’une configuration performante. En combinant ces deux critères, l’optimisation bayésienne cherche à réduire l’incertitude en explorant les régions peu explorées tout en exploitant les régions près d’une configuration performante. C’est ce qu’on appelle une fonction d’acquisition

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from pprint import pprint

In [ ]:
# Define search space
search_spaces = {
     'learning_rate': np.arange(0.01, 1.0),
     'max_depth': np.arange(2, 20),
     'reg_lambda': np.arange(1e-9, 100),
     'reg_alpha': np.arange(1e-9, 100),
     'gamma': np.arange(1e-9, 0.5),  
     'n_estimators': np.arange(10, 5000)
}

In [ ]:
# Create Bayesian CV for HP optimization
bayes_cv = BayesSearchCV(
                    estimator = model,                                    
                    search_spaces = search_spaces,                      
                    scoring = 'roc_auc',                                  
                    cv = 5,                                   
                    n_iter = 20,                                      
                    n_points = 5,                                       
                    n_jobs = -1,                                        
                    refit=False,
                    verbose = 1,
                    random_state=42
)        

In [ ]:
# Run bayesian CV
%time bayes_cv.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
Fitting 5 folds for each of 5 candidates, totalling 25 fits
CPU times: user 46.1 s, sys: 9.92 s, total: 56 s
Wall time: 1min 9s


BayesSearchCV(cv=5,
              estimator=XGBClassifier(base_score=None, booster='gbtree',
                                      callbacks=None, colsample_bylevel=None,
                                      colsample_bynode=None,
                                      colsample_bytree=None, device=None,
                                      early_stopping_rounds=None,
                                      enable_categorical=True,
                                      eval_metric='auc', feature_types=None,
                                      gamma=None, grow_policy=None,
                                      importance_type=None,
                                      interaction_constraints=None,
                                      learning_rate=...
       5.6e+01, 5.7e+01, 5.8e+01, 5.9e+01, 6.0e+01, 6.1e+01, 6.2e+01,
       6.3e+01, 6.4e+01, 6.5e+01, 6.6e+01, 6.7e+01, 6.8e+01, 6.9e+01,
       7.0e+01, 7.1e+01, 7.2e+01, 7.3e+01, 7.4e+01, 7.5e+01, 7.6e+01,
       7.7e+01, 7.8e+01, 7.9e+01, 8.0e+01, 8.1e+01, 8.2e+01, 8.3e+01,
       8.4e+01, 8.5e+01, 8.6e+01, 8.7e+01, 8.8e+01, 8.9e+01, 9.0e+01,
       9.1e+01, 9.2e+01, 9.3e+01, 9.4e+01, 9.5e+01, 9.6e+01, 9.7e+01,
       9.8e+01, 9.9e+01])},
              verbose=1)

In [ ]:
# Show best params
print('Best parameters:')
print("Best score = %.3f after %d runs" % (bayes_cv.best_score_, bayes_cv.n_iter))

Best parameters:
Best score = 0.626 after 30 runs


Best score = 0.630 after 20 runs

Best score = 0.638 after 300 runs - 35min 27s 

##### Option 4: Approche 'HalvingRandomSearchCV' & 'HalvingGridSearchCV' ( to document)
Permet d'explorer le champs des possibles sans forcément trouver la combinaison optimale. L'avantage que présente cette méthode est de pouvoir maitriser le temps de calcul.
Chercher des paramètres sur des échantillons de plus en plus important en écartant a chaque itération la moitiée des combinaisons les moins performantes.

In [36]:
from sklearn.model_selection import HalvingGridSearchCV 
from sklearn.model_selection import HalvingRandomSearchCV 

In [53]:

param_dist = {
    'learning_rate': [0.1, 0.05, 0.3, 0.2, 0.4 ,0.6, 0.7, 0.01],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
}


In [54]:
halving_grid = HalvingGridSearchCV(model,
                           param_grid= param_dist, 
                           cv=5, 
                           scoring='roc_auc',
                           n_jobs=-1, 
                           random_state=42,
                           verbose=1)

In [55]:
# Exécuter la recherche d'hyperparamètres
halving_grid.fit(X_train, y_train)



n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 1022
max_resources_: 27612
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 72
n_resources: 1022
Fitting 5 folds for each of 72 candidates, totalling 360 fits
----------
iter: 1
n_candidates: 24
n_resources: 3066
Fitting 5 folds for each of 24 candidates, totalling 120 fits
----------
iter: 2
n_candidates: 8
n_resources: 9198
Fitting 5 folds for each of 8 candidates, totalling 40 fits
----------
iter: 3
n_candidates: 3
n_resources: 27594
Fitting 5 folds for each of 3 candidates, totalling 15 fits


HalvingGridSearchCV(estimator=XGBClassifier(base_score=None, booster='gbtree',
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=True,
                                            eval_metric='auc',
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate...
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=-1,
                                            num_parallel_tree=None,
                                            random_state=None, ...),
                    n_jobs=-1,
                    param_grid={'learning_rate': [0.1, 0.05, 0.3, 0.2, 0.4, 0.6,
                                                  0.7, 0.01],
                                'max_depth': [3, 5, 7],
                                'n_estimators': [100, 200, 300]},
                    random_state=42, scoring='roc_auc', verbose=1)

In [56]:
# Afficher les performances des meilleurs paramètres
score = halving_grid.score(X, y)
print(score)

0.774562370316416


##### Option 5 : SMAC (Sequential Model-based Algorithm Configuration) - Forets aléatoires  (to do)

SMAC est un framework d'optimisation basé sur des modèles séquentiels. Il est principalement utilisé pour automatiser la recherche des meilleurs hyperparamètres pour un algorithme d'apprentissage automatique donné. SMAC utilise des modèles statistiques pour modéliser la relation entre les hyperparamètres et la fonction de coût. Il recherche les hyperparamètres les plus prometteurs en utilisant une approche séquentielle, en essayant de minimiser la fonction de coût prédite par le modèle.
SMAC équilibre l'exploitation (exploration des valeurs déjà connues prometteuses) et l'exploration (essai de nouvelles valeurs) pour rechercher efficacement l'espace des hyperparamètres.

##### Option 6: Hyperopt, Optuna, gpyopt (to do)
